In [149]:
from yahooquery import Ticker
from datetime import date
import pandas as pd
from IPython.display import display

symbols = ['fb', 'amzn', 'aapl', 'nflx', 'goog']
faang = Ticker(symbols)
data = []
history = faang.history(period='ytd', interval='1d', end=date.today())

for symbol in symbols:
    data.append(history.loc[str(symbol)][['open','high','low','close','volume']].rename(columns={'open':'Open','high':'High'
                                                                                                 ,'low':'Low','close':'Close','volume':'Volume'}))

In [159]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

class SmaCross(Strategy):
    def init(self):
        n1 = 10
        n2 = 40
        price = self.data.Close
        self.ma1 = self.I(SMA, price, n1)
        self.ma2 = self.I(SMA, price, n2)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()

In [160]:
for i, symbol in enumerate(symbols):
    bt = Backtest(data[i], SmaCross, commission=.002,
                  exclusive_orders=True, cash=10_000)
    stats = bt.run()
    bt.plot()

<ipython-input-160-8a2635e766b4>:2: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(data[i], SmaCross, commission=.002,
